# Function Attributes

До сих пор мы имели дело с невызываемыми атрибутами. Когда атрибуты на самом деле являются функциями - они ведет себя по-другому.

In [1]:
class Person:
    def say_hello():
        print('Hello!')

In [2]:
Person.say_hello

<function __main__.Person.say_hello()>

In [3]:
type(Person.say_hello)

function

Как мы видим, это просто простая функция, которая вызывается как обычно:

In [4]:
Person.say_hello()

Hello!


Теперь давайте создадим экземпляр класса `Person` и посмотрим на атрибут `say_hello` с точки зрения этого экземпляра.

In [5]:
p = Person()

In [6]:
hex(id(p))

'0x7f88a06937b8'

Мы знаем, что можем **получить доступ к атрибутам класса через экземпляр**, поэтому мы также должны иметь возможность получить доступ к атрибуту функции таким же образом:

In [7]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x7f88a06937b8>>

In [8]:
type(p.say_hello)

method

Хм, тип изменился с `function` на `method`, а представление функции гласит, что это **связанный метод** **конкретного объекта** `p`, который мы создали (обратите внимание на адрес памяти).

А если мы попытаемся вызвать функцию из экземпляра, вот что произойдет:

In [9]:
try:
    p.say_hello()
except Exception as ex:
    print(type(ex).__name__, ex)

TypeError say_hello() takes 0 positional arguments but 1 was given


`method` является фактическим типом в Python. Как и в случаи функции, экземпляр этого класса являются вызываемым, но у него есть одна отличительная черта. Его необходимо привязать к объекту, и эта ссылка на этот объект передается базовой функции в качестве первого позиционного аргумента.

Часто, когда мы определяем функции в классе и вызываем их из экземпляра, нам нужно знать, какой **конкретный** экземпляр использовался для вызова функции. Это позволяет нам взаимодействовать с переменными экземпляра.

Для этого Python автоматически преобразует обычную функцию, определенную в классе, в метод, когда она вызывается из экземпляра класса.

Кроме того, он «привязывает» метод к экземпляру, то есть экземпляр будет передан как **первый** аргумент вызываемой функции.

Он делает это с помощью **дескрипторов**, к которым мы вернемся подробно позже.

А пока давайте просто рассмотрим это немного подробнее:

In [19]:
class Person:
    def say_hello(*args):
        print('say_hello args:', args)

In [2]:
Person.say_hello()

say_hello args: ()


Как мы видим, вызов `say_hello` из **класса** просто вызывает функцию (это просто функция).

Но когда мы вызываем ее из экземпляра:

In [3]:
p = Person()    # создим экземпляр класса
hex(id(p))      # м посмотрим его идентификатор

'0x7f24d50f5610'

In [8]:
p.say_hello()   # обратимся к атрибут-функции класса через созданный экземпляр

say_hello args: (<__main__.Person object at 0x7f24d50f5610>,)


Вы можете видеть, что объект `p` был передан как аргумент функции класса `say_hello`.


In [21]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'say_hello': <function __main__.Person.say_hello(*args)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [22]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x7f24cc43eff0>>

---


Очевидным преимуществом преобразования функции класса в метод экземпляра класса и соответственно передачи ссылки на экземпляр в качестве первого аргумента при обращенн к методу, является то, что теперь мы можем легко взаимодействовать с атрибутами экземпляра:

В примере ниже - мы используем метод для инициализации атрибута экземпляра (установки значения)

In [14]:
class Person:
    def set_name(instance_obj, new_name):
        instance_obj.name = new_name  # or setattr(instance_obj, 'name', new_name)


In [15]:
p = Person()

In [16]:
p.set_name('Alex')

In [17]:

p.__dict__

{'name': 'Alex'}

По сути, это имеет тот же эффект, что и следующее:

In [18]:
Person.set_name(p, 'John')

In [19]:
p.__dict__

{'name': 'John'}

>По соглашению первый аргумент обычно называется `self`, но, как вы только что видели, мы можем назвать его как угодно.

---

Короче говоря, функции, определенные в классе, преобразуются в методы при вызове из экземпляров класса. Поэтому, конечно, нам нужно учитывать этот дополнительный аргумент, который передается методу.

---

**Метод** — это тип, экземпляр которого создаваемые Python при вызове функций класса из экземпляра этого класс.

У методов есть свои собственные уникальные атрибуты:

In [11]:
class Person:
    def say_hello(self):
        print(f'{self} says hello')

In [12]:
p = Person()

In [13]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x7f24cc43eff0>>

Посмотрим атрибуты метода

In [14]:
m_hello = p.say_hello

In [15]:
type(m_hello)

method

Например, у него есть атрибут `__func__`:

In [16]:
m_hello.__func__

<function __main__.Person.say_hello(self)>

которая является функцией класса, используемой для создания метода (базовой функции).

In [17]:
m_hello.__func__ is Person.say_hello

True

У функции класса и метода экземпляра отличается состав атрибутов

In [9]:
dir(p.say_hello)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [10]:
dir(Person.say_hello)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__type_params__']

Но помните, что метод привязан к экземпляру. В этом случае мы получили метод из объекта `p`:

In [26]:
hex(id(p))

'0x7f88d0428c18'

In [27]:
m_hello.__self__

Как вы можете видеть, метод также имеет ссылку на объект, к которому он **привязан**.

Поэтому думайте о методах как о функциях, которые были привязаны к определенному объекту, и этот объект передается в качестве первого аргумента вызова функции. Остальные аргументы затем передаются после этого.

Методы экземпляра создаются автоматически для нас, когда мы определяем функции внутри определений наших классов.

На этапе описания класса say_hello - это еще не метод - это функция, которая учитывает, что ее будут вызывать из экземпляра класса. Методом эта функция становится когда будет создан экземпляр класса и эта функция будет с этим экземпляром связана (т.е. получит ссылку на экземпляр класса)

Пространство имен класса и пространство имен экземпляра класса полностью изолированы друг от друга. Когда мы описываем функцию в классе, мы должны учитывать, что функции не нужны сами по себе (чаще всего), а нужны для обработки тех состояний , которые сохранены в экземпляре класса. Но класс не имеет доступ к пространству имен своего экземпляра. Единственный способ получить доступ к  сохраненному состоянию - это получить ссылку на сам экземпляр. Именно поэтому Python делает неявную передачу экземпляра класса в функции класса (методы) в первом позиционном параметре `self` - чтобы получить доступ к пространству имен экземпляра класса из самих классов.

---

Это все справедливо даже если мы применяем `Monkey paching` к нашим классам во время выполнения (Примечание ниже):

In [23]:
class Person:
    def say_hello(self):
        print(f'instance method called from {self}')

In [24]:
p = Person()
hex(id(p))

'0x7f24cc46c5f0'

In [25]:
p.say_hello()

instance method called from <__main__.Person object at 0x7f24cc46c5f0>


In [26]:
Person.do_work = lambda self: f"do_work called from {self}"

In [27]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'say_hello': <function __main__.Person.say_hello(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'do_work': <function __main__.<lambda>(self)>})

Хорошо, обе функции находятся в словаре (пространстве имен) `__dict__`.

давайте создадим экземпляр и посмотрим, что произойдет:

In [28]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x7f24cc46c5f0>>

In [29]:
p.do_work

<bound method <lambda> of <__main__.Person object at 0x7f24cc46c5f0>>

In [35]:
p.do_work()

'do_work called from <__main__.Person object at 0x7f88d0435f28>'

Но будьте осторожны, если мы добавим функцию к **экземпляру** напрямую, это будет работать не так — мы создадим функцию в экземпляре, поэтому она не будет считаться методом (так как она не была определена в классе):

In [36]:
p.other_func = lambda *args: print(f'other_func called with {args}')

In [37]:
p.other_func

<function __main__.<lambda>(*args)>

In [38]:
'other_func' in Person.__dict__

False

In [39]:
p.other_func()

other_func called with ()


Как видите, `other_func` является и ведет себя как обычная функция.

Короче говоря, функции, определенные в классе, преобразуются в методы при вызове из экземпляров класса. Поэтому, конечно, мы должны учитывать этот дополнительный аргумент, который передается методу.

---

## Лирическое отступление


### Monkey paching

`Monkey patching` в контексте `Python` — это техника, позволяющая изменять или расширять поведение классов или модулей во время выполнения программы.
Это достигается путем динамического изменения атрибутов, методов или функций, что позволяет вносить изменения в уже существующий код без необходимости его модификации.

### Основные аспекты monkey patching:

1. **Изменение поведения**: `Monkey patching` позволяет изменять поведение существующих функций или методов. Это может быть полезно для исправления ошибок, добавления новой функциональности или изменения логики работы.

2. **Динамическое изменение**: Патчинг происходит во время выполнения программы, что позволяет вносить изменения в код, который уже был загружен в память.

3. **Использование**: `Monkey patching` часто используется в тестировании, когда необходимо заменить определенные методы или функции на заглушки (mock) для изоляции тестируемого кода. Также его можно использовать для исправления ошибок в сторонних библиотеках, если нет возможности изменить исходный код.

Пример monkey patching:

In [ ]:
class MyClass:
    def greet(self):
        return "Hello!"

# Исходное поведение
obj = MyClass()
print(obj.greet())  # Вывод: Hello!

# Monkey patching: изменяем метод greet
def new_greet(self):
    return "Hi there!"

MyClass.greet = new_greet

# Теперь поведение изменилось
print(obj.greet())  # Вывод: Hi there!

- **Неочевидность**: `Monkey patching` может сделать код менее понятным и предсказуемым, так как изменения происходят динамически и могут быть неочевидны для других разработчиков.
- **Проблемы с совместимостью**: Если библиотека или класс обновляются, изменения, внесенные с помощью `monkey patching`, могут перестать работать или вызвать неожиданные ошибки.
- **Тестирование**: Хотя `monkey patching` может быть полезен в тестах, его использование должно быть ограничено, чтобы избежать сложностей в поддержке тестов.

---
## Пример


In [1]:
import sys

from typing import Callable
from types import FunctionType, MethodType
from pprint import pprint

In [2]:
class Person:
    '''Супер-класс'''

    def say_hello() -> None:
        print('Hello')


In [3]:


def func() -> None:
    '''
    Пример фунции, определенной
    в глобальном пространстве имён
    '''

    print("Global Func")

In [5]:
# Что находится в пространстве имен модуля
current_module = sys.modules[__name__]

pprint({
    name: value
    for name, value
    in current_module.__dict__.items()
    if name != '__builtins__'
})

{'Callable': typing.Callable,
 'FunctionType': <class 'function'>,
 'In': ['',
        'import sys\n'
        '\n'
        'from typing import Callable\n'
        'from types import FunctionType, MethodType\n'
        'from pprint import pprint',
        'class Person:\n'
        "    '''Супер-класс'''\n"
        '\n'
        '    def say_hello() -> None:\n'
        "        print('Hello')",
        'def func() -> None:\n'
        "    '''\n"
        '    Пример фунции, определенной\n'
        '    в глобальном пространстве имён\n'
        "    '''\n"
        '\n'
        '    print("Global Func")',
        '# Что находится в пространстве имен модуля\n'
        'current_module = sys.modules[__name__]\n'
        '\n'
        'pprint({\n'
        '    name: value\n'
        '    for name, value\n'
        '    in current_module.__dict__.items()\n'
        "    if name != '__builtins__'\n"
        '})',
        '# Что находится в пространстве имен модуля\n'
        'current_module = sys.m

In [6]:
# Получить коллекцию callable-объектов, определенных
# в глобальном пространстве имен
# ... множество разнообразных объектов и типов
pprint({
	name: value
	for name, value
	in current_module.__dict__.items()
	if callable(value)
})

{'Callable': typing.Callable,
 'FunctionType': <class 'function'>,
 'MethodType': <class 'method'>,
 'Person': <class '__main__.Person'>,
 'exit': <IPython.core.autocall.ZMQExitAutocall object at 0x7f36985b0e30>,
 'func': <function func at 0x7f36985bafc0>,
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f36a13717c0>>,
 'open': <function open at 0x7f36a20094e0>,
 'pprint': <function pprint at 0x7f36a2a33880>,
 'quit': <IPython.core.autocall.ZMQExitAutocall object at 0x7f36985b0e30>}


In [7]:
# Получить коллекцию "чистых" ф-ций, определенных
# в глобальном пространстве имен
pprint({
    name: value
    for name, value
    in current_module.__dict__.items()
    if isinstance(value, FunctionType)
})

{'func': <function func at 0x7f36985bafc0>,
 'open': <function open at 0x7f36a20094e0>,
 'pprint': <function pprint at 0x7f36a2a33880>}


In [8]:
# Протестируем функцию func определенную в пространстве имен модуля
# (глобальное пространство имен)
# Смоделируем такую же строку ...
print(
    f"<{type(func).__name__} "
    f"{func.__name__} "
    f"at {hex(id(func))}>"
)

<function func at 0x7f36985bafc0>


In [ ]:
# Вызов функции, объявленной в пространстве имён класс Person
Person.say_hello()

In [9]:
# Связывание объекта-функции определенной в пространстве имен класса
# с глобальной переменной
f = Person.say_hello
pprint({
	name: value
	for name, value
	in globals().items()
	if not callable(value) and not name.startswith("__")
})


{'In': ['',
        'import sys\n'
        '\n'
        'from typing import Callable\n'
        'from types import FunctionType, MethodType\n'
        'from pprint import pprint',
        'class Person:\n'
        "    '''Супер-класс'''\n"
        '\n'
        '    def say_hello() -> None:\n'
        "        print('Hello')",
        'def func() -> None:\n'
        "    '''\n"
        '    Пример фунции, определенной\n'
        '    в глобальном пространстве имён\n'
        "    '''\n"
        '\n'
        '    print("Global Func")',
        '# Что находится в пространстве имен модуля\n'
        'current_module = sys.modules[__name__]\n'
        '\n'
        'pprint({\n'
        '    name: value\n'
        '    for name, value\n'
        '    in current_module.__dict__.items()\n'
        "    if name != '__builtins__'\n"
        '})',
        '# Что находится в пространстве имен модуля\n'
        'current_module = sys.modules[__name__]\n'
        '\n'
        'pprint({\n'
        '    

In [10]:
print(hex(id(func)))
f()
print(f'type(Person.say_hello) = {type(Person.say_hello)}')
print(f'type(f) = {type(f)}')
print(f'id(Person.say_hello) = {hex(id(Person.say_hello))}')
print(f'id(f) = {hex(id(f))}')

0x7f36985bafc0
Hello
type(Person.say_hello) = <class 'function'>
type(f) = <class 'function'>
id(Person.say_hello) = 0x7f36985b9d00
id(f) = 0x7f36985b9d00
